In [ ]:
"""
	Autores:
	- Junior Lara
	- Astrid Alvarado
"""

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, col, lower, avg, round, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType
import pycountry

In [2]:
spark = SparkSession\
    .builder\
    .appName("Solver problema 1")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/28 20:56:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [22]:
# Para esta primera parte basta con inferencia para extraer la data necesaria.
# Se necesitan los siguientes campos
# - Mal ID
# - Location

file = "../tests/vshort-users-details-2023.csv"

df_user_location = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

df_user_location.show(5)

+------+--------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|Mal ID|Username|Gender|            Birthday|            Location|              Joined|Days Watched|Mean Score|Watching|Completed|On Hold|Dropped|Plan to Watch|Total Entries|Rewatched|Episodes Watched|
+------+--------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|     1|   Xinil|  Male|1985-03-04T00:00:...|          California|2004-11-05T00:00:...|       142.3|      7.37|     1.0|    233.0|    8.0|   93.0|         64.0|        399.0|     60.0|          8458.0|
|     3| Aokaado|  Male|                NULL|        Oslo, Norway|2004-11-11T00:00:...|        68.6|      7.34|    23.0|    137.0|   99.0|   44.0|         40.0|        343.0|     15.0|        

In [23]:
# Por DataFrame
DF_id_location = df_user_location.select("Mal ID", "Location")

In [24]:
# Por SQL

# Se genera la VIEW
df_user_location.createOrReplaceTempView("df_user_location")

# Se ejecuta la consulta
SQL_id_location = spark.sql("SELECT `Mal ID`, Location FROM df_user_location")

In [28]:
# Mapeo manual de abreviaciones de subdivisiones a países
ABBREVIATION_MAP = {
    "ab": "Canada",
    "bc": "Canada",
    "on": "Canada",
    "qc": "Canada",
    "ny": "United States",
    "ca": "United States",
    "tx": "United States",
    "usa": "United States",
    "uk": "United Kingdom",
    "uae": "United Arab Emirates",
    "prc": "China",
    "us": "United States",
    "gb": "United Kingdom",
    "ru": "Russia",
    "au": "Australia",
    "nz": "New Zealand",
    "de": "Germany",
    "fr": "France",
    "es": "Spain",
    "it": "Italy",
    "jp": "Japan",
    "cn": "China",
    "kr": "South Korea",
    "br": "Brazil",
    "mx": "Mexico",
    "in": "India",
    "za": "South Africa",
    "se": "Sweden",
    "no": "Norway",
    "fi": "Finland",
    "dk": "Denmark",
    "ch": "Switzerland",
    "nl": "Netherlands",
    "be": "Belgium",
    "at": "Austria",
    "pl": "Poland",
    "ar": "Argentina",
    "cl": "Chile",
    "co": "Colombia",
    "ve": "Venezuela",
    "pe": "Peru",
    "ph": "Philippines",
    "id": "Indonesia",
    "my": "Malaysia",
    "sg": "Singapore",
    "th": "Thailand",
    "vn": "Vietnam",
    "sa": "Saudi Arabia",
    "eg": "Egypt",
    "tr": "Turkey",
    "gr": "Greece",
    "pt": "Portugal",
    "cz": "Czech Republic",
    "sk": "Slovakia",
    "hu": "Hungary",
    "ro": "Romania",
    "bg": "Bulgaria",
    "rs": "Serbia",
    "hr": "Croatia",
    "si": "Slovenia",
    "ua": "Ukraine",
    "by": "Belarus",
    "lt": "Lithuania",
    "lv": "Latvia",
    "ee": "Estonia",
    "is": "Iceland",
    "ie": "Ireland",
    "pk": "Pakistan",
    "bd": "Bangladesh",
    "lk": "Sri Lanka",
    "np": "Nepal",
    "af": "Afghanistan",
    "ir": "Iran",
    "iq": "Iraq",
    "sy": "Syria",
    "jo": "Jordan",
    "lb": "Lebanon",
    "il": "Israel",
    "ps": "Palestine",
    "kw": "Kuwait",
    "qa": "Qatar",
    "bh": "Bahrain",
    "om": "Oman",
    "ye": "Yemen",
    "dz": "Algeria",
    "ma": "Morocco",
    "tn": "Tunisia",
    "ly": "Libya",
    "sd": "Sudan",
    "ng": "Nigeria",
    "gh": "Ghana",
    "ke": "Kenya",
    "tz": "Tanzania",
    "ug": "Uganda",
    "zm": "Zambia",
    "zw": "Zimbabwe",
    "bw": "Botswana",
    "na": "Namibia",
    "ao": "Angola",
    "mz": "Mozambique",
    "mg": "Madagascar",
    "et": "Ethiopia",
    "sn": "Senegal",
    "ci": "Ivory Coast",
    "cm": "Cameroon",
    "cd": "Congo",
    "cg": "Republic Of The Congo",
    "ga": "Gabon",
    "gq": "Equatorial Guinea",
    "cv": "Cape Verde",
    "st": "Sao Tome And Principe",
    "sc": "Seychelles",
    "mu": "Mauritius",
    "km": "Comoros",
    "dj": "Djibouti",
    "er": "Eritrea",
    "so": "Somalia",
    "rw": "Rwanda",
    "bi": "Burundi",
    "mw": "Malawi",
    "ls": "Lesotho",
    "sz": "Eswatini",
    "sl": "Sierra Leone",
    "lr": "Liberia",
    "gm": "Gambia",
    "gw": "Guinea-Bissau",
    "gn": "Guinea",
    "ml": "Mali",
    "bf": "Burkina Faso",
    "ne": "Niger",
    "td": "Chad",
    "mr": "Mauritania",
    "bj": "Benin",
    "tg": "Togo",
    "cf": "Central African Republic",
    "ss": "South Sudan",
    "bt": "Bhutan",
    "mv": "Maldives",
    "kh": "Cambodia",
    "la": "Laos",
    "mm": "Myanmar",
    "bn": "Brunei",
    "tl": "Timor-Leste",
    "pg": "Papua New Guinea",
    "fj": "Fiji",
    "ws": "Samoa",
    "to": "Tonga",
    "vu": "Vanuatu",
    "sb": "Solomon Islands",
    "ki": "Kiribati",
    "tv": "Tuvalu",
    "nr": "Nauru",
    "pw": "Palau",
    "mh": "Marshall Islands",
    "fm": "Micronesia",
    "as": "American Samoa",
    "gu": "Guam",
    "mp": "Northern Mariana Islands",
    "ck": "Cook Islands",
    "nu": "Niue",
    "wf": "Wallis And Futuna",
    "pf": "French Polynesia",
    "nc": "New Caledonia",
    "tk": "Tokelau",
    "pn": "Pitcairn Islands",
    "gs": "South Georgia And The South Sandwich Islands",
    "sh": "Saint Helena",
    "fk": "Falkland Islands",
    "ai": "Anguilla",
    "bm": "Bermuda",
    "vg": "British Virgin Islands",
    "ky": "Cayman Islands",
    "ms": "Montserrat",
    "tc": "Turks And Caicos Islands",
    "vi": "US Virgin Islands",
    "pr": "Puerto Rico",
    "um": "United States Minor Outlying Islands",
    "hk": "Hong Kong",
    "mo": "Macau",
    "tw": "Taiwan",
    "fo": "Faroe Islands",
    "gl": "Greenland",
    "ax": "Aland Islands",
    "je": "Jersey",
    "gg": "Guernsey",
    "im": "Isle Of Man",
    "yt": "Mayotte",
    "re": "Reunion",
    "mq": "Martinique",
    "gp": "Guadeloupe",
    "bl": "Saint Barthelemy",
    "mf": "Saint Martin",
    "pm": "Saint Pierre And Miquelon",
    "tf": "French Southern Territories",
    "bv": "Bouvet Island",
    "hm": "Heard Island And Mcdonald Islands",
    "aq": "Antarctica",
    "cw": "Curacao",
    "sx": "Sint Maarten",
    "bq": "Caribbean Netherlands",
    "xk": "Kosovo"
}

# Función para identificar el país a partir de la ubicación
def get_country(location):
    if not location or location.strip() == "":
        return "UNKNOWN"
    
    location = location.lower().strip()
    location_parts = [part.strip() for part in location.split(",")]  # Dividir por comas y limpiar espacios
    
    # Verificar si alguna parte coincide con una abreviación en el mapeo manual
    for part in location_parts:
        if part in ABBREVIATION_MAP:
            return ABBREVIATION_MAP[part]  # Devolver el país correspondiente
    
    # Verificar si el país ya está incluido en la ubicación
    for country in pycountry.countries:
        if country.name.lower() in location_parts or \
           country.alpha_2.lower() in location_parts or \
           country.alpha_3.lower() in location_parts:
            return country.name
    
    # Verificar subdivisiones (estados, provincias, etc.)
    for subdivision in pycountry.subdivisions:
        if subdivision.name.lower() in location_parts:
            return pycountry.countries.get(alpha_2=subdivision.country_code).name
    
    # Si no se encuentra, devolver "UNKNOWN"
    return "UNKNOWN"

In [ ]:
# Por DataFrame

# Registrar la función como UDF
get_country_udf = udf(get_country, StringType())

# Normalizar la columna Location y aplicar la función para obtener el país
DF_id_location_with_country = DF_id_location.withColumn("Country", get_country_udf(lower(col("Location"))))

# Seleccionar solo las columnas Mal ID y Country
DF_id_country = DF_id_location_with_country.select("Mal ID", "Country")

# Mostrar los resultados
DF_id_country.show(10, truncate=False)

+------+-------------+
|Mal ID|Country      |
+------+-------------+
|1     |United States|
|3     |Norway       |
|4     |Australia    |
|9     |UNKNOWN      |
|18    |UNKNOWN      |
|20    |Norway       |
|23    |Canada       |
|36    |UNKNOWN      |
|44    |UNKNOWN      |
|47    |UNKNOWN      |
+------+-------------+
only showing top 10 rows



In [29]:
# Por SQL

# Registrar la función UDF en SQL
spark.udf.register("get_country_sql", get_country, StringType())

# Crear una vista temporal para el DataFrame id_location
DF_id_location.createOrReplaceTempView("DF_id_location")

# Consulta SQL para aplicar la lógica de mapeo y seleccionar Mal ID y Country
SQL_id_country = spark.sql("""
    SELECT 
        `Mal ID`,
        get_country_sql(Location) AS Country
    FROM 
        id_location
""")

# Mostrar los resultados
SQL_id_country.show(10, truncate=False)

25/03/28 21:10:55 WARN SimpleFunctionRegistry: The function get_country_sql replaced a previously registered function.


+------+-------------+
|Mal ID|Country      |
+------+-------------+
|1     |United States|
|3     |Norway       |
|4     |Australia    |
|9     |UNKNOWN      |
|18    |UNKNOWN      |
|20    |Norway       |
|23    |Canada       |
|36    |UNKNOWN      |
|44    |UNKNOWN      |
|47    |UNKNOWN      |
+------+-------------+
only showing top 10 rows



In [30]:
# Se quiere extraer las puntaciones que realizaron los usuarios.
# Se necesitan los siguientes campos
# - user_id
# - rating

file = "../tests/vshort-users-score-2023.csv"

df_user_rating = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

df_user_rating.show(5)

+-------+--------+--------+--------------------+------+
|user_id|Username|anime_id|         Anime Title|rating|
+-------+--------+--------+--------------------+------+
|      1|   Xinil|      21|           One Piece|     9|
|      1|   Xinil|      48|         .hack//Sign|     7|
|      1|   Xinil|     320|              A Kite|     5|
|      1|   Xinil|      49|    Aa! Megami-sama!|     8|
|      1|   Xinil|     304|Aa! Megami-sama! ...|     8|
+-------+--------+--------+--------------------+------+
only showing top 5 rows



In [33]:
# Por DataFrame

# Calcular la media de los ratings por usuario y redondear a 2 decimales
DF_average_ratings = df_user_rating.groupBy("user_id").agg(round(avg("rating"), 2).alias("average_rating"))

# Mostrar los resultados
DF_average_ratings.show(10, truncate=False)

+-------+--------------+
|user_id|average_rating|
+-------+--------------+
|23     |7.46          |
|9      |7.71          |
|1      |7.44          |
|20     |8.06          |
|37     |7.0           |
|4      |6.52          |
|83     |5.0           |
+-------+--------------+



In [34]:
# Por SQL

# Crear una vista temporal del DataFrame
df_user_rating.createOrReplaceTempView("ratings")

# Ejecutar la consulta SQL para calcular el promedio redondeado a 2 decimales
SQL_average_ratings = spark.sql("""
    SELECT 
        user_id, 
        ROUND(AVG(rating), 2) AS average_rating
    FROM ratings
    GROUP BY user_id
""")

# Mostrar los resultados
SQL_average_ratings.show(10, truncate=False)

+-------+--------------+
|user_id|average_rating|
+-------+--------------+
|23     |7.46          |
|9      |7.71          |
|1      |7.44          |
|20     |8.06          |
|37     |7.0           |
|4      |6.52          |
|83     |5.0           |
+-------+--------------+



In [43]:
# Renombrar la columna "Mal ID" a "user_id" en id_location_with_country_final
DF_id_country = DF_id_country.withColumnRenamed("Mal ID", "user_id")

In [54]:
# Por DataFrame

# Realizar el join entre DF_average_ratings e DF_id_country
DF_join_avg_country = DF_average_ratings.join(DF_id_country, on="user_id", how="inner")

# Calcular el promedio de los ratings por país y la cantidad de usuarios
DF_avg_ratings_by_country = DF_join_avg_country.groupBy("Country").agg(
    round(avg("average_rating"), 2).alias("average"),
    count("user_id").alias("user_count")
)

# Mostrar los resultados
DF_avg_ratings_by_country.show(10, truncate=False)

+-------------+-------+----------+
|Country      |average|user_count|
+-------------+-------+----------+
|United States|6.22   |2         |
|Norway       |8.06   |1         |
|UNKNOWN      |7.71   |1         |
|Canada       |7.46   |1         |
|Australia    |6.52   |1         |
+-------------+-------+----------+



In [55]:
# EN SQL

# Crear vistas temporales para los DataFrames
DF_average_ratings.createOrReplaceTempView("DF_average_ratings")
DF_id_country.createOrReplaceTempView("DF_id_country")

# Consulta SQL para realizar el JOIN, calcular el promedio por país y contar la cantidad de usuarios
SQL_avg_ratings_by_country = spark.sql("""
    SELECT 
        c.Country,
        ROUND(AVG(a.average_rating), 2) AS average,
        COUNT(a.user_id) AS user_count
    FROM 
        DF_average_ratings a
    INNER JOIN 
        DF_id_country c
    ON 
        a.user_id = c.user_id
    GROUP BY 
        c.Country
""")

# Mostrar los resultados
SQL_avg_ratings_by_country.show(20, truncate=False)

+-------------+-------+----------+
|Country      |average|user_count|
+-------------+-------+----------+
|United States|6.22   |2         |
|Norway       |8.06   |1         |
|UNKNOWN      |7.71   |1         |
|Canada       |7.46   |1         |
|Australia    |6.52   |1         |
+-------------+-------+----------+



In [ ]:
# Por DataFrame

# Renombrar la columna "Mal ID" a "user_id" en DF_id_country
DF_id_country = DF_id_country.withColumnRenamed("Mal ID", "user_id")

# Ordenar por user_count de mayor a menor y seleccionar el top 10
DF_top_10_countries = DF_avg_ratings_by_country.orderBy(col("user_count").desc()).limit(10)

# Dividir el top 10 en dos grupos: mejores y peores promedios
DF_top_5_best_averages = DF_top_10_countries.orderBy(col("average").desc()).limit(5)
DF_top_5_worst_averages = DF_top_10_countries.orderBy(col("average").asc()).limit(5)

# Mostrar los resultados
print("Top 5 países con mejores promedios:")
DF_top_5_best_averages.show(truncate=False)

print("Top 5 países con peores promedios:")
DF_top_5_worst_averages.show(truncate=False)

Top 5 países con mejores promedios:
+-------------+-------+----------+
|Country      |average|user_count|
+-------------+-------+----------+
|Norway       |8.06   |1         |
|UNKNOWN      |7.71   |1         |
|Canada       |7.46   |1         |
|Australia    |6.52   |1         |
|United States|6.22   |2         |
+-------------+-------+----------+

Top 5 países con peores promedios:
+-------------+-------+----------+
|Country      |average|user_count|
+-------------+-------+----------+
|United States|6.22   |2         |
|Australia    |6.52   |1         |
|Canada       |7.46   |1         |
|UNKNOWN      |7.71   |1         |
|Norway       |8.06   |1         |
+-------------+-------+----------+



In [ ]:
# Por SQL

# Crear una vista temporal para SQL_avg_ratings_by_country
SQL_avg_ratings_by_country.createOrReplaceTempView("SQL_avg_ratings_by_country")

# Obtener el top 10 de países con más usuarios
SQL_top_10_countries = spark.sql("""
    SELECT *
    FROM SQL_avg_ratings_by_country
    ORDER BY user_count DESC
    LIMIT 10
""")
SQL_top_10_countries.createOrReplaceTempView("top_10_countries")

# Obtener los 5 países con mejores promedios del top 10
SQL_top_5_best_averages = spark.sql("""
    SELECT *
    FROM top_10_countries
    ORDER BY average DESC
    LIMIT 5
""")

# Obtener los 5 países con peores promedios del top 10
SQL_top_5_worst_averages = spark.sql("""
    SELECT *
    FROM top_10_countries
    ORDER BY average ASC
    LIMIT 5
""")

# Mostrar los resultados
print("Top 5 países con mejores promedios:")
SQL_top_5_best_averages.show(truncate=False)

print("Top 5 países con peores promedios:")
SQL_top_5_worst_averages.show(truncate=False)

Top 5 países con mejores promedios:


+-------------+-------+----------+
|Country      |average|user_count|
+-------------+-------+----------+
|Norway       |8.06   |1         |
|UNKNOWN      |7.71   |1         |
|Canada       |7.46   |1         |
|Australia    |6.52   |1         |
|United States|6.22   |2         |
+-------------+-------+----------+

Top 5 países con peores promedios:
+-------------+-------+----------+
|Country      |average|user_count|
+-------------+-------+----------+
|United States|6.22   |2         |
|Australia    |6.52   |1         |
|Canada       |7.46   |1         |
|UNKNOWN      |7.71   |1         |
|Norway       |8.06   |1         |
+-------------+-------+----------+



In [59]:
spark.stop()